In [1]:
# Area de Importación 

In [2]:
import pandas
import sqlite3

In [3]:
# Conexión al archivo SQLITE para obtener info y variable a usar para
# crear los DataFrames

In [4]:
con = sqlite3.connect("database.sqlite")

In [5]:
# Se guarda cada tabla del DB en una variable para su manipulación individual. 

In [6]:
Country = pandas.read_sql("SELECT * from Country", con)
League = pandas.read_sql("SELECT * from League", con)
Match = pandas.read_sql("SELECT * from Match", con)
Player = pandas.read_sql("SELECT * from Player", con)
Player_Attributes = pandas.read_sql("SELECT * from Player_Attributes", con)
Team = pandas.read_sql("SELECT * from Team", con)
Team_Attributes = pandas.read_sql("SELECT * from Team_Attributes", con)

In [7]:
# PANDAS TIME!! Volvemos las variables que guardan nuestras querys,
# en fans de Pxndx, usando "pandas.DataFrame(variable_tabla)"

#           "¿Te acuerdas cuando me dijiste: "dificil se pondrá"? 
#           $Python fue lo que pasó,
#           y prometiste que cuando yo muera, guardarías mi corazóoon.to_csv <3"


In [8]:
df_country = pandas.DataFrame(Country)
df_league = pandas.DataFrame(League)

In [9]:
# Una vez revisado ambos dataframes por separado, se decide unir ambas tablas, ya que tienen información del pais y la liga,
# así como el id, que mas adelante se usará para completar la tabla "Match/Partido"

In [10]:
df_country_league = df_country.merge(right=df_league, left_on='id',right_on='id').rename(columns={'id':'id','name_x':'Country','country_id':'Country_id','name_y':'Ligue'}).drop(['id'],1)

In [11]:
# En la tabla de los partidos, se concentra la información de los juegos, jugadores y datos estadísticos, pero usando
# id's de los jugaores, equipos y pais/liga. Por lo que se crean 2 diccionarios a partir de la información de liga, pais
# con su llave. Cabe mencionar que el Id es el mismo para el pais y la liga.

In [12]:
dic_ligues = dict(zip(df_country_league.Country_id, df_country_league.Ligue))
dic_countries = dict(zip(df_country_league.Country_id, df_country_league.Country))

def substitute_league(x):
    for k,v in dic_ligues.items():
        if x == k:
            return v

def substitute_country(x):
    for k,v in dic_countries.items():
        if x == k:
            return v
df_country_league

,Country,Country_id,Ligue
0,Belgium,1,Belgium Jupiler League
1,England,1729,England Premier League
2,France,4769,France Ligue 1
3,Germany,7809,Germany 1. Bundesliga
4,Italy,10257,Italy Serie A
5,Netherlands,13274,Netherlands Eredivisie
6,Poland,15722,Poland Ekstraklasa
7,Portugal,17642,Portugal Liga ZON Sagres
8,Scotland,19694,Scotland Premier League
9,Spain,21518,Spain LIGA BBVA


In [13]:
# Se decide usar esta tabla como pivote para unificar los datos que contienen las demas tablas de la base, pero utilizando
# los nombres de equipos, jugadores, liga y país en el que se desarrolló un partido.
# Adicional se retiran las columnas que mas información basura contenian.

In [14]:
df_match = pandas.DataFrame(Match)
Match_DMD = df_match.dropna().drop(['goal','shoton','shotoff', 'foulcommit','card','cross','corner','possession'], 1)
Match_DMD

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
3248,3249,1729,1729,2012/2013,1,2012-08-18 00:00:00,1228229,9825,8472,0,...,9.50,1.44,4.75,8.50,1.35,4.60,9.00,1.40,4.33,8.50
3249,3250,1729,1729,2012/2013,1,2012-08-20 00:00:00,1228230,8668,10260,1,...,1.91,4.30,3.70,1.87,4.60,3.50,1.83,4.00,3.40,1.91
3250,3251,1729,1729,2012/2013,1,2012-08-18 00:00:00,1228231,9879,9850,5,...,4.50,1.83,3.75,4.75,1.80,3.50,4.40,1.83,3.50,4.33
3251,3252,1729,1729,2012/2013,1,2012-08-19 00:00:00,1228232,8456,8466,3,...,19.00,1.20,7.00,19.00,1.18,6.25,16.00,1.17,6.50,17.00
3252,3253,1729,1729,2012/2013,1,2012-08-18 00:00:00,1228233,10261,8586,2,...,2.80,2.62,3.40,2.75,2.60,3.30,2.75,2.50,3.40,2.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23412,23413,21518,21518,2012/2013,9,2012-10-28 00:00:00,1260116,8581,7878,3,...,3.80,2.05,3.50,3.90,2.00,3.20,3.90,2.00,3.40,3.50
23413,23414,21518,21518,2012/2013,9,2012-10-28 00:00:00,1260118,8394,8302,2,...,2.38,3.20,3.40,2.38,3.30,3.20,2.35,2.88,3.30,2.38
23414,23415,21518,21518,2012/2013,9,2012-10-28 00:00:00,1260121,8661,8633,0,...,1.33,9.00,5.25,1.36,8.75,4.75,1.40,8.50,4.50,1.36
23415,23416,21518,21518,2012/2013,9,2012-10-27 00:00:00,1260123,9910,9783,1,...,3.80,1.95,3.60,4.20,1.90,3.40,4.00,1.91,3.50,3.75


In [15]:
# Se unifican las tablas de jugador y los atributos de jugadores, y se agrupan por las columnas en las que se repiten 
# para dejar 1 sola fila para capturar toda la información de 1 partido.
# Tambien se crea el diccionario para modificar los valores de la tabla principal a usar (Match), igual se parsea un dato 
# para hacerlo compatible con la misma tabla.

In [16]:
df_player = pandas.DataFrame(Player)
df_player_att = pandas.DataFrame(Player_Attributes)

DAP = df_player.merge(right=df_player_att, on='player_api_id').dropna().drop(['id_x',
                            'id_y','date','preferred_foot','attacking_work_rate','defensive_work_rate'],1)

GroupDAP = DAP.groupby(['player_api_id', 'player_name', 'player_fifa_api_id_x',
                        'birthday','height', 'weight',
                        'player_fifa_api_id_y']).mean().reset_index().round(2)

dic_players = dict(zip(df_player.player_api_id, df_player.player_name))

def substitute_player(x):
    for k,v in dic_players.items():
        if x == k:
            return v

GroupDAP['height'].astype('int64')
GroupDAP

,player_api_id,player_name,player_fifa_api_id_x,birthday,height,weight,player_fifa_api_id_y,overall_rating,potential,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,2625,"Patryk Rachwal,18",148544,1981-01-27 00:00:00,175.26,154,148544,60.14,61.14,50.14,...,55.29,61.93,62.93,63.86,56.57,12.43,12.86,19.07,10.36,10.43
1,2752,Diego Mainz,135819,1982-12-29 00:00:00,187.96,174,135819,69.38,70.38,36.43,...,52.43,35.43,71.10,70.67,65.57,11.10,6.10,7.10,9.10,15.10
2,2768,Jose Dorado,110019,1982-07-10 00:00:00,180.34,154,110019,69.29,70.57,42.24,...,57.76,31.90,71.67,70.62,79.71,10.76,15.90,22.71,15.14,12.10
3,2770,Ignacio Gonzalez,182861,1982-05-14 00:00:00,180.34,161,182861,71.13,73.53,61.87,...,71.60,63.93,30.20,37.53,25.00,8.67,15.67,27.20,16.33,17.00
4,2790,Alberto Rey,110809,1974-02-15 00:00:00,175.26,163,110809,70.20,75.80,70.00,...,60.00,55.40,74.60,74.60,69.00,8.00,17.00,59.00,16.60,17.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10412,744907,Greg Taylor,226508,1997-11-05 00:00:00,172.72,137,226508,51.91,65.91,42.45,...,51.45,52.45,39.45,48.45,49.45,8.45,13.45,12.45,11.45,8.45
10413,746419,Francisco Afonso,233930,1997-04-24 00:00:00,175.26,152,233930,59.00,66.00,55.00,...,40.00,43.00,55.00,69.00,62.00,7.00,10.00,7.00,9.00,9.00
10414,748432,Fynn Arkenberg,233969,1996-03-04 00:00:00,187.96,176,233969,58.00,68.00,48.00,...,36.00,45.00,63.00,69.00,68.00,8.00,8.00,12.00,12.00,6.00
10415,750435,Rees Greenwood,225462,1996-01-20 00:00:00,172.72,150,225462,56.44,70.44,35.00,...,58.00,61.00,18.00,19.00,21.00,9.00,10.00,8.00,10.00,11.00


In [17]:
#Se realiza limpieza por separado de las tablas de equipo y los atributos del equipo, así como las llaves que identifican
# los valores dentro de dicha tabla principa y antes de agruparlas, se agrupa a si misma para reducir la cantidad de renglones.

In [18]:
df_team = pandas.DataFrame(Team).dropna().drop(['team_fifa_api_id'], axis=1) 
df_team_att = pandas.DataFrame(Team_Attributes).drop(['buildUpPlayDribbling'],1)
Group_team_att = df_team_att.groupby(['team_fifa_api_id']).mean().reset_index().round(2)
Team = df_team.merge(right=Group_team_att, on='team_api_id')
GroupTeam = Team.groupby(['team_api_id','team_long_name','team_short_name']).mean().round(2).reset_index()
GroupTeam

C:\Users\fpedrero\AppData\Roaming\Python\Python37\site-packages\pandas\core\reshape\merge.py:1100: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,


,team_api_id,team_long_name,team_short_name,id_x,team_fifa_api_id,id_y,buildUpPlaySpeed,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
0,1601,Ruch Chorzów,CHO,31446,874,1115.5,47.33,46.17,55.50,60.33,53.50,47.17,47.33,49.33
1,1773,Oud-Heverlee Leuven,O-H,1513,100087,942.0,52.00,50.00,50.00,53.00,54.00,43.00,44.00,50.00
2,1957,Jagiellonia Białystok,BIA,31456,110745,621.5,47.17,52.67,50.83,36.33,67.00,49.33,56.33,53.67
3,2033,S.C. Olhanense,OLH,35774,111540,930.0,49.00,48.40,51.80,44.20,48.20,42.20,33.80,45.40
4,2182,Lech Poznań,POZ,31453,873,697.5,60.33,49.00,56.67,57.33,68.00,51.67,48.50,54.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,10278,Hércules Club de Fútbol,HER,44557,100879,577.0,50.60,49.40,56.60,49.80,61.80,46.60,49.80,55.40
278,10281,Real Valladolid,VAL,43050,462,1322.5,52.83,53.67,50.33,59.17,50.33,45.50,51.17,52.67
279,108893,AC Arles-Avignon,ARL,11074,111989,62.5,43.17,53.17,46.00,38.33,39.50,38.33,46.00,46.33
280,158085,FC Arouca,ARO,37696,112513,69.5,67.00,39.00,55.00,59.00,46.00,36.00,38.00,37.00


In [19]:
Match_event = Match_DMD.merge(df_country_league, left_on='country_id', right_on='Country_id')
Match_event.drop(Match_event.columns[11:55], inplace=True, axis=1)


dic = dict(zip(df_team.team_api_id,df_team.team_long_name))

def substitute(x):
    for k,v in dic.items():
        if x == k:
            return v
Match_event['home_team_api_id'] = Match_event['home_team_api_id'].apply(substitute)
Match_event['away_team_api_id'] = Match_event['away_team_api_id'].apply(substitute)
Match_event['country_id'] = Match_event['country_id'].apply(substitute_country)
Match_event['league_id'] = Match_event['league_id'].apply(substitute_league)

#Match_event.columns[11:33] = Match_event.columns[11:33]
for i in Match_event.columns[11:33]:
    Match_event[i] = Match_event[i].apply(substitute_player)
    
Match_event.to_csv('match_Event.csv', index=False)
Match_event

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,VCA,GBH,GBD,GBA,BSH,BSD,BSA,Country,Country_id,Ligue
0,3249,England,England Premier League,2012/2013,1,2012-08-18 00:00:00,1228229,Arsenal,Sunderland,0,...,8.50,1.35,4.60,9.00,1.40,4.33,8.50,England,1729,England Premier League
1,3250,England,England Premier League,2012/2013,1,2012-08-20 00:00:00,1228230,Everton,Manchester United,1,...,1.87,4.60,3.50,1.83,4.00,3.40,1.91,England,1729,England Premier League
2,3251,England,England Premier League,2012/2013,1,2012-08-18 00:00:00,1228231,Fulham,Norwich City,5,...,4.75,1.80,3.50,4.40,1.83,3.50,4.33,England,1729,England Premier League
3,3252,England,England Premier League,2012/2013,1,2012-08-19 00:00:00,1228232,Manchester City,Southampton,3,...,19.00,1.18,6.25,16.00,1.17,6.50,17.00,England,1729,England Premier League
4,3253,England,England Premier League,2012/2013,1,2012-08-18 00:00:00,1228233,Newcastle United,Tottenham Hotspur,2,...,2.75,2.60,3.30,2.75,2.50,3.40,2.70,England,1729,England Premier League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757,23413,Spain,Spain LIGA BBVA,2012/2013,9,2012-10-28 00:00:00,1260116,Levante UD,Granada CF,3,...,3.90,2.00,3.20,3.90,2.00,3.40,3.50,Spain,21518,Spain LIGA BBVA
1758,23414,Spain,Spain LIGA BBVA,2012/2013,9,2012-10-28 00:00:00,1260118,Real Zaragoza,Sevilla FC,2,...,2.38,3.30,3.20,2.35,2.88,3.30,2.38,Spain,21518,Spain LIGA BBVA
1759,23415,Spain,Spain LIGA BBVA,2012/2013,9,2012-10-28 00:00:00,1260121,RCD Mallorca,Real Madrid CF,0,...,1.36,8.75,4.75,1.40,8.50,4.50,1.36,Spain,21518,Spain LIGA BBVA
1760,23416,Spain,Spain LIGA BBVA,2012/2013,9,2012-10-27 00:00:00,1260123,RC Celta de Vigo,RC Deportivo de La Coruña,1,...,4.20,1.90,3.40,4.00,1.91,3.50,3.75,Spain,21518,Spain LIGA BBVA
